In [ ]:
# --- Dependencies ---
import os
import re
import json
import pandas as pd

# Try to import pdfplumber when needed
try:
    import pdfplumber
except Exception as e:
    pdfplumber = None  # we only require this if input is a PDF


# --- Helpers ---
def parse_price(cell):
    """Extract an integer Naira amount from strings like '₦69,000.00', '15,000', '15000.5', etc."""
    if cell is None:
        return None
    s = str(cell).strip()
    if not s or s in {"?", "N/A", "NA", "-", "--"}:
        return None
    s = s.replace("₦", "").replace("\u20a6", "").replace(",", "").strip()
    m = re.search(r"(\d+(?:\.\d+)?)", s)
    if not m:
        return None
    try:
        return int(round(float(m.group(1))))
    except Exception:
        return None


def is_upper_heading(text):
    """Heuristic for category rows: looks like uppercase words (&,/,- allowed), no digits."""
    if not text:
        return False
    t = str(text)
    if re.search(r"\d", t):
        return False
    cleaned = re.sub(r"[^A-Za-z/&\-\s]", "", t).strip()
    return bool(cleaned) and cleaned.upper() == cleaned and len(cleaned) >= 3


def clean_cells(row):
    """Trim all cells to simple strings (or None)."""
    out = []
    for c in row:
        if c is None:
            out.append(None)
        else:
            s = str(c).strip()
            out.append(s if s != "" else None)
    return out


# --- CSV path (legacy support, preserved from your original) ---
def csv_to_records(csv_file):
    df = pd.read_csv(csv_file, header=None, dtype=str)

    candidate_cols = [c for c in df.columns if c != 0]

    def numeric_score(series):
        s = series.dropna().astype(str)
        return (s.str.contains(r"\d").sum()) / max(len(s), 1)

    price_col = max(candidate_cols, key=lambda c: numeric_score(df[c])) if candidate_cols else None

    data = []
    current_category = None

    for _, row in df.iterrows():
        name = str(row.get(0)) if pd.notna(row.get(0)) else ""

        # Category row (uppercase in col 0, others empty)
        others_empty = all(pd.isna(row.get(c)) or str(row.get(c)).strip() == "" for c in [1, 2, 3] if c in df.columns)
        if name and others_empty and name.strip().upper() == name.strip() and name.strip() != "TEST":
            current_category = name.strip().title()
            continue

        if name.strip().upper() in {"TEST", "", "NA"}:
            continue

        if name.strip():
            price = parse_price(row.get(price_col)) if price_col is not None else None
            if price is not None or current_category is not None:
                data.append({
                    "category": current_category,
                    "test": name.strip(),   # preserve original case (keeps acronyms)
                    "price": price
                })

    return data


# --- PDF path ---
def extract_rows_from_pdf(pdf_path):
    """Return a list of row-lists from the PDF. Tries table extraction, then falls back to text split."""
    if pdfplumber is None:
        raise ImportError(
            "pdfplumber is required for PDF parsing. Install with: pip install pdfplumber"
        )

    rows = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # Try tables via line-based detection, then default
            tables = (page.extract_tables(table_settings={
                "vertical_strategy": "lines",
                "horizontal_strategy": "lines",
                "intersection_y_tolerance": 5,
                "snap_tolerance": 6,
                "text_x_tolerance": 2,
                "text_y_tolerance": 2,
            }) or page.extract_tables())

            if tables:
                for table in tables:
                    for raw in table:
                        if not raw:
                            continue
                        row = clean_cells(raw)
                        # Skip obvious header rows
                        joined = " ".join([c for c in row if c])
                        if re.search(r"\b(TEST|INVESTIGATION|PRICE|AMOUNT|FEE|COST)\b", joined, re.I):
                            continue
                        rows.append(row)
            else:
                # Fallback: split text lines by 2+ spaces (common in pricelists)
                text = page.extract_text() or ""
                for line in text.splitlines():
                    if not line.strip():
                        continue
                    # Skip page headers/footers if any pattern fits
                    if re.search(r'^(Page\s*\d+|\d+\s*/\s*\d+)$', line, re.I):
                        continue
                    parts = re.split(r"\s{2,}", line.strip())
                    if parts:
                        rows.append(clean_cells(parts))

    return rows


def rows_to_records(rows):
    """Convert generic rows into structured records with category/test/price."""
    data = []
    current_category = None

    for row in rows:
        # Trim empty tail
        while row and (row[-1] is None or str(row[-1]).strip() == ""):
            row.pop()
        if not row:
            continue

        non_empty = [c for c in row if c and str(c).strip()]

        # Category: single non-empty cell that looks like a heading
        if len(non_empty) == 1 and is_upper_heading(non_empty[0]):
            current_category = str(non_empty[0]).strip().title()
            continue

        # Find rightmost price-like cell
        price = None
        price_idx = None
        for i in range(len(row) - 1, -1, -1):
            p = parse_price(row[i])
            if p is not None:
                price = p
                price_idx = i
                break

        # First non-empty cell not equal to the price cell is the test name
        name = None
        for i, c in enumerate(row):
            if i == price_idx:
                continue
            if c and str(c).strip():
                name = str(c).strip()
                break

        if not name:
            continue

        # Skip obvious headers
        if name.strip().upper() in {"TEST", "INVESTIGATION", "S/N", "SN"}:
            continue

        data.append({
            "category": current_category,
            "test": name,   # preserve acronyms / original case
            "price": price
        })

    return data


def pdf_to_records(pdf_file):
    rows = extract_rows_from_pdf(pdf_file)
    return rows_to_records(rows)


# --- Unified entrypoint ---
def file_to_json(input_file, json_file):
    ext = os.path.splitext(input_file)[1].lower()
    if ext in {".csv", ".tsv"}:
        records = csv_to_records(input_file)
    elif ext == ".pdf":
        records = pdf_to_records(input_file)
    else:
        raise ValueError(f"Unsupported file type: {ext}")

    with open(json_file, "w", encoding="utf-8") as f:
        json.dump(records, f, ensure_ascii=False, indent=2)

    print(f"✅ Wrote {len(records)} tests to {json_file}")


# ------- Run it (adjust the filename as needed) -------
# Example:
# file_to_json("EPICONSULT_PRICELIST_2025.pdf", "tests.json")


In [ ]:
# Example:
# file_to_json("EPICONSULT_PRICELIST_2025.pdf", "tests.json")